In [10]:
import pandas as pd

sentiment_data = pd.read_excel("C:/Users/Getzbie Alfredo Tpoy/Documents/MSIB MARIBELAJAR/Capstone/FIX DATASET.xlsx")

In [11]:
sentiment_data.head()

,Deskripsi,Label
0,Baik,1
1,aman terkendali,1
2,baik,1
3,Gampang,1
4,administrasi sangat baik karena perminggunya d...,1


### Lower case Converting

In [12]:
sentiment_data["Deskripsi"] = sentiment_data["Deskripsi"].str.lower()

In [13]:
print("Lower case Converting :\n")
print(sentiment_data['Deskripsi'].head(10))
print("\n\n")

Lower case Converting :

0                                                 baik
1                                      aman terkendali
2                                                 baik
3                                              gampang
4    administrasi sangat baik karena perminggunya d...
5    administrasi cukup baik karena dituntuk juga o...
6                                                 baik
7    terjun langsung dan merasakan menjadi karyawan...
8                                                bagus
9    pembelajarannya sangat berguna namun proses be...
Name: Deskripsi, dtype: object





### Tokenizing, dan Cleansing

In [14]:
import string 
import re

# gunakan nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist


def remove_Reviews_special(text):
    # mengahapus tab, line , dan backslash
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # hapus non ASCII
    text = text.encode('ascii', 'replace').decode('ascii')
    # hapus mention
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # hapus url
    return text.replace("http://", " ").replace("https://", " ")
                
sentiment_data['Deskripsi'] = sentiment_data['Deskripsi'].apply(remove_Reviews_special)

# hapus angka
def remove_number(text):
    return  re.sub(r"\d+", "", text)

sentiment_data['Deskripsi'] = sentiment_data['Deskripsi'].apply(remove_number)

# hapus punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

sentiment_data['Deskripsi'] = sentiment_data['Deskripsi'].apply(remove_punctuation)

# hapus multiple spasi
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

sentiment_data['Deskripsi'] = sentiment_data['Deskripsi'].apply(remove_whitespace_multiple)

# hapus single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

sentiment_data['Deskripsi'] = sentiment_data['Deskripsi'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

sentiment_data['Deskripsi_tokens'] = sentiment_data['Deskripsi'].apply(word_tokenize_wrapper)

print('Hasil Tokenizing : \n') 
print(sentiment_data['Deskripsi_tokens'].head())
print('\n\n')

Hasil Tokenizing : 

0                                               [baik]
1                                   [aman, terkendali]
2                                               [baik]
3                                            [gampang]
4    [administrasi, sangat, baik, karena, perminggu...
Name: Deskripsi_tokens, dtype: object





### Stopwords removal

In [15]:
from nltk.corpus import stopwords

#stopword indonesia dari nltk
list_stopwords = stopwords.words('indonesian')

#txt stopword
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string ke list
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# convert ke dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

sentiment_data['Deskripsi_Stopword'] = sentiment_data['Deskripsi_tokens'].apply(stopwords_removal) 


print(sentiment_data['Deskripsi_Stopword'].head())

0                                                   []
1                                   [aman, terkendali]
2                                                   []
3                                            [gampang]
4    [administrasi, perminggunya, diadakan, pertemu...
Name: Deskripsi_Stopword, dtype: object


### Stemm

In [17]:
# import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in sentiment_data['Deskripsi_Stopword']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmeer ke dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

sentiment_data['Deskripsi_tokens_stemmed'] = sentiment_data['Deskripsi_Stopword'].swifter.apply(get_stemmed_term)
print(sentiment_data['Deskripsi_tokens_stemmed'])

314
------------------------
aman : aman
terkendali : kendali
gampang : gampang
administrasi : administrasi
perminggunya : minggu
diadakan : ada
pertemuan : temu
khusus : khusus
membahas : bahas
laporan : lapor
bulanan : bulan
dituntuk : dituntuk
facilitator : facilitator
mentor : mentor
terjun : terjun
langsung : langsung
merasakan : rasa
karyawan : karyawan
pengalaman : alam
pelajaran : ajar
dapatkan : dapat
bagus : bagus
pembelajarannya : ajar
berguna : guna
proses : proses
belajarnya : ajar
lumayan : lumayan
membosankan : bosan
hehehehe : hehehehe
asisten : asisten
ramah : ramah
perizinan : izin
berhalangan : halang
kegiatan : giat
bertabrakan : tabrak
kampus : kampus
seru : seru
interaktif : interaktif
tergantung : gantung
struggle : struggle
lancar : lancar
task : task
sehari : hari
anak : anak
intern : intern
dilepas : lepas
pengerjaannya : kerja
good : good
dashboard : dashboard
seri : seri
tahap : tahap
pengerjaan : kerja
berkolaborasi : kolaborasi
kelas : kelas
capstone : cap

Pandas Apply:   0%|          | 0/93 [00:00<?, ?it/s]

0                                                    []
1                                       [aman, kendali]
2                                                    []
3                                             [gampang]
4     [administrasi, minggu, ada, temu, khusus, baha...
                            ...                        
88                                              [error]
89                  [reschedule, mentor, kena, musibah]
90                             [sulit, diskusi, luring]
91    [lumayan, sulit, rekognisi, sks, fokus, progra...
92         [lumayan, sulit, implementasi, coding, mula]
Name: Deskripsi_tokens_stemmed, Length: 93, dtype: object


In [18]:
sentiment_data.to_excel("Sentiment_Preprocessing.xlsx")